Faydalandığımız kaynak
- https://www.kaggle.com/code/akshitrai/chatbot-jarvis
- https://scikit-learn.org/stable/auto_examples/model_selection/grid_search_text_feature_extraction.html

In [1]:
import os
import json
import numpy as np 
import string
from unicode_tr import unicode_tr
from nltk.corpus import stopwords
from snowballstemmer import TurkishStemmer
from nltk import ngrams
import pandas as pd 
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import TfidfTransformer,TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from time import time

In [2]:
path = r'..\veri'

def yukle():
    liste = os.listdir(path)
    final_path = path + "\\" + liste[-1]
    print(final_path)
    with open(final_path, encoding='utf-8') as f:
        veri = json.load(f)
    return veri

In [3]:
df_ = yukle()

..\veri\ankarabbSoruCevapVeriSetiJson-v12-onislenmis.json


In [4]:
questions, responses = [],[]


for veri_ in df_:
    for i, j in enumerate(df_[veri_]):
        for k,l in enumerate(j["patterns"]):
            questions.append(l)
            responses.append(df_[veri_][i]["responses"][0])
        # for k,l in enumerate(j["responses"]):
        #     responses.append(l)

In [5]:
df = pd.DataFrame([questions,responses]).T

In [6]:
df

,0,1
0,öğrenci abonma kart yararlanma şart nelerdir? ...,1. Ankara Büyükşehir Belediye Başkanımız Sayın...
1,öğrenci iç abonma kart şart ne öğrenciiç içabo...,1. Ankara Büyükşehir Belediye Başkanımız Sayın...
2,abonma kart öğrenci şart abonmakart kartöğrenc...,1. Ankara Büyükşehir Belediye Başkanımız Sayın...
3,abonma kart iç öğrenci ne yapmal abonmakart ka...,1. Ankara Büyükşehir Belediye Başkanımız Sayın...
4,indirimli abonma kart iç öğrenci yapmas gereke...,1. Ankara Büyükşehir Belediye Başkanımız Sayın...
...,...,...
853,atıksu bedel ne atıksubedel bedelne,Ankara ili sınırları içinde idaremize ait kana...
854,atıksu bedel ne alınır atıksubedel bedelne nea...,Ankara ili sınırları içinde idaremize ait kana...
855,atıksu bedel hukuki dayanak ne atıksubedel bed...,Ankara ili sınırları içinde idaremize ait kana...
856,atıksu bedel ne zama alınma başla atıksubedel ...,Ankara ili sınırları içinde idaremize ait kana...


In [ ]:
# Define a pipeline combining a text feature extractor with a simple classifier
pipeline = Pipeline(
    [
        ("vect", CountVectorizer()),
        ("tfidf", TfidfTransformer()),
        ("clf", DecisionTreeClassifier()),
    ]
)

# Parameters to use for grid search. Uncommenting more parameters will give
# better exploring power but will increase processing time in a combinatorial
# way
parameters = {
    "vect__max_df": (0.5, 0.75, 1.0),
    'vect__max_features': (None, 5000, 10000, 50000),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    "clf__n_estimators": (20,50,100,150, 200),
    "clf__criterion" : ("gini", "entropy", "log_loss"),
    "clf__max_depth" : ("None", 2, 3, 5),
    "clf__min_samples_leaf" : (1, 3, 5),
    "clf__max_features" : ("sqrt", "log2", None)
}

# Find the best parameters for both the feature extraction and the
# classifier
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1)

print("Performing grid search...")
print("pipeline:", [name for name, _ in pipeline.steps])
print("parameters:")
pprint(parameters)
t0 = time()
grid_search.fit(df[0],df[1])
print("done in %0.3fs" % (time() - t0))
print()

print("Best score: %0.3f" % grid_search.best_score_)
print("Best parameters set:")
best_parameters = grid_search.best_estimator_.get_params()
for param_name in sorted(parameters.keys()):
    print("\t%s: %r" % (param_name, best_parameters[param_name]))

In [7]:
DecisionTreeClassifierModel = Pipeline([
    ('bow',CountVectorizer()),
    ('tfidf',TfidfTransformer()),
    ('classifier', DecisionTreeClassifier())
])

In [8]:
DecisionTreeClassifierModel.fit(df[0],df[1])

Pipeline(steps=[('bow', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('classifier', DecisionTreeClassifier())])

In [9]:
def metin_onisle(metin):
    kucuk_harfli_metin = unicode_tr(metin).lower()
    istenen_karakterler = set(list(' abcdefghijklmnopqrstuvwxyzâçîöüğış0123456789'))
    harfler = list(kucuk_harfli_metin)
    harfler = [k if k in istenen_karakterler else ' ' for k in harfler]
    temiz_dokuman = "".join(kucuk_harfli_metin)
    temiz_dokuman_kelimeleri = temiz_dokuman.split(' ')
    temiz_dokuman_kelimeleri = [kelime for kelime in temiz_dokuman_kelimeleri if len(kelime) > 0]
    turkStem=TurkishStemmer()
    temiz_dokuman_kelimeleri = [turkStem.stemWord(kelime) for kelime in temiz_dokuman_kelimeleri]
    n = 2
    bigrams = ngrams(temiz_dokuman_kelimeleri, n)
    bigramstr = map(''.join, bigrams)
    ngram = " ".join(list(bigramstr))
    temiz_dokuman = " ".join(temiz_dokuman_kelimeleri) + " " +  "".join(ngram)
    return temiz_dokuman

In [10]:
DecisionTreeClassifierModel.predict([metin_onisle("Evlendirme İşlemleri Hizmetleri bilgi")])[0]

'Evlendirme Şube Müdürlüğü evlenmek için başvuruda bulunan Türk vatandaşı ve Yabancı uyruklu kişilerin evlilik başvurularını almakta ve Gençlik Parkı içerisinde 1 adet oda nikah salonunda oda nikahı hizmeti ve Ankara mücavir alan sınırları içerinde bulunan çiftlerin belirlediği mekanlara nikah memuru göndererek nikah hizmeti vermektedir.'

In [11]:
DecisionTreeClassifierModel.predict([metin_onisle("Başkent Kartın Aidat Ücreti")])[0]

'Türkiye’nin en kapsamlı şehir kartı bir çok alanda hayatınızı kolaylaştırırken, siz de herhangi bir kart aidatı ödemezsiniz.'

In [16]:
DecisionTreeClassifierModel.predict_proba([metin_onisle("Başkent Kartın Aidat Ücreti")])

array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]])

In [21]:
len(DecisionTreeClassifierModel.predict_proba([metin_onisle("Başkent Kartın Aidat Ücreti")])[0])

172

AttributeError: 'CountVectorizer' object has no attribute 'n_features_in_'

In [ ]:
DecisionTreeClassifierModel.predict([metin_onisle("Cenaze / defin işlemi")])[0]

In [ ]:
DecisionTreeClassifierModel.predict([metin_onisle("Öğrencilerin ücretsiz taşındığı hat")])[0]

In [ ]:
import pickle

with open('../ankarabbHibritChatbot/DecisionTreeClassifierModel.pickle', 'wb') as handle:
    pickle.dump(DecisionTreeClassifierModel, handle, protocol=pickle.HIGHEST_PROTOCOL)
    